In [1]:
DRIVE_BASE = "/content/drive/MyDrive/PTBXL"  # path inside their mounted Drive


In [2]:
import os
DRIVE_BASE = "/content/drive/MyDrive/PTBXL"   # change if you want a different folder
os.makedirs(DRIVE_BASE, exist_ok=True)
print("Workspace folder:", DRIVE_BASE)

# Where the PTBXL files will live inside Drive (physionet wget saves into nested folders)
PTBXL_DRIVE_PATH = os.path.join(DRIVE_BASE, "physionet.org/files/ptb-xl/1.0.3/")
print("Expected PTB-XL root in Drive:", PTBXL_DRIVE_PATH)


Workspace folder: /content/drive/MyDrive/PTBXL
Expected PTB-XL root in Drive: /content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/


In [3]:
# run as shell cell in Colab
!pip install wfdb==4.3.0 pandas numpy scikit-learn tqdm scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [5]:
from google.colab import drive
import os

# 1. Force disconnect the drive if it's stuck
try:
    drive.flush_and_unmount()
    print("Unmounted successfully.")
except Exception:
    pass

# 2. Delete the stuck folder
!rm -rf /content/drive

# 3. Remount fresh
print("Remounting...")
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Unmounted successfully.
Remounting...
Mounted at /content/drive


In [6]:
!wget -r -N -c -np -P "/content/drive/MyDrive/PTBXL/" https://physionet.org/files/ptb-xl/1.0.3/


Streaming output truncated to the last 5000 lines.

--2025-12-22 06:06:46--  https://physionet.org/files/ptb-xl/1.0.3/records100/00000/00284_lr.dat
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/records100/00000/00284_lr.dat’ not modified on server. Omitting download.

--2025-12-22 06:06:46--  https://physionet.org/files/ptb-xl/1.0.3/records100/00000/00284_lr.hea
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/records100/00000/00284_lr.hea’ not modified on server. Omitting download.

--2025-12-22 06:06:46--  https://physionet.org/files/ptb-xl/1.0.3/records100/00000/00285_lr.dat
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/content/drive/MyDrive/PTBXL/physionet.org/file

In [7]:
!pkill -f "wget"


In [8]:
%%bash
rm -rf "/content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/records500"


In [9]:
import os
DRIVE_BASE = "/content/drive/MyDrive/PTBXL"   # change if you used a different folder name
PTBXL_ROOT = os.path.join(DRIVE_BASE, "physionet.org/files/ptb-xl/1.0.3/")
print("PTBXL_ROOT =", PTBXL_ROOT)
print("Exists:", os.path.exists(PTBXL_ROOT))
print("Listing top-level:", sorted(os.listdir(PTBXL_ROOT))[:20])
# Ensure records100 and ptbxl_database.csv exist
assert os.path.exists(os.path.join(PTBXL_ROOT, "ptbxl_database.csv")), "ptbxl_database.csv not found!"
assert os.path.exists(os.path.join(PTBXL_ROOT, "records100")), "records100 folder not found!"


PTBXL_ROOT = /content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/
Exists: True
Listing top-level: ['LICENSE.txt', 'RECORDS', 'SHA256SUMS.txt', 'example_physionet.py', 'index.html', 'ptbxl_database.csv', 'ptbxl_v102_changelog.txt', 'ptbxl_v103_changelog.txt', 'records100', 'scp_statements.csv']


In [10]:
import pandas as pd, ast, os, numpy as np
db_path = os.path.join(PTBXL_ROOT, "ptbxl_database.csv")
scp_path = os.path.join(PTBXL_ROOT, "scp_statements.csv")

df = pd.read_csv(db_path)
scp_df = pd.read_csv(scp_path)

# convert stringified dict to actual dict
df["scp_codes"] = df["scp_codes"].apply(ast.literal_eval)

print("Rows in metadata:", len(df))
print("Columns:", df.columns.tolist())
df.head(2)


Rows in metadata: 21799
Columns: ['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis', 'infarction_stadium1', 'infarction_stadium2', 'validated_by', 'second_opinion', 'initial_autogenerated_report', 'validated_by_human', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr']


,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr


In [11]:
import os, pandas as pd
PTBXL_ROOT = "/content/drive/MyDrive/PTBXL/physionet.org/files/ptb-xl/1.0.3/"
db_path = os.path.join(PTBXL_ROOT, "ptbxl_database.csv")
df = pd.read_csv(db_path)
print("Columns in CSV:", df.columns.tolist())
print("\nSample rows (first 3):")
display(df.head(3))

# Confirm 100Hz filenames exist
has_lr = "filename_lr" in df.columns
has_hr = "filename_hr" in df.columns
print("filename_lr present:", has_lr, "  filename_hr present:", has_hr)

# Quick counts
print("Total records in CSV:", len(df))
print("Total files in records100 folder (approx):",
      len([p for p in os.listdir(os.path.join(PTBXL_ROOT, "records100")) if p.endswith(".dat")]) )


Columns in CSV: ['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis', 'infarction_stadium1', 'infarction_stadium2', 'validated_by', 'second_opinion', 'initial_autogenerated_report', 'validated_by_human', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr']

Sample rows (first 3):


,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr


filename_lr present: True   filename_hr present: True
Total records in CSV: 21799
Total files in records100 folder (approx): 0


In [12]:
# Safe parser for scp_codes column
import ast, json, pandas as pd, os

def safe_parse_scp(x):
    # already a dict
    if isinstance(x, dict):
        return x
    # missing
    if pd.isna(x):
        return {}
    # if bytes
    if isinstance(x, bytes):
        try:
            x = x.decode('utf-8')
        except:
            return {}
    # strings
    if isinstance(x, str):
        s = x.strip()
        if s == "" or s.lower() in ("nan", "none", "null"):
            return {}
        # Some rows may already look like Python dicts with single quotes (good for ast)
        try:
            return ast.literal_eval(s)
        except Exception:
            # Try relaxed JSON: convert single quotes -> double quotes and attempt json.loads
            try:
                j = s.replace("'", '"')
                return json.loads(j)
            except Exception:
                # Last resort: try to recover small formatting issues (remove non-ascii weird chars)
                try:
                    # remove control characters
                    cleaned = ''.join(ch for ch in s if ord(ch) >= 32)
                    cleaned = cleaned.replace("'", '"')
                    return json.loads(cleaned)
                except Exception:
                    # give up -> return a placeholder dict but record original
                    return {"__PARSE_ERROR__": s}
    # unknown types
    return {}

# Apply parser and report problematic rows
bad_rows = []
parsed = []
for idx, val in df["scp_codes"].items():
    parsed_val = safe_parse_scp(val)
    parsed.append(parsed_val)
    if isinstance(parsed_val, dict) and "__PARSE_ERROR__" in parsed_val:
        bad_rows.append((idx, parsed_val["__PARSE_ERROR__"]))

df["scp_codes_parsed"] = parsed
print("Total rows:", len(df))
print("Parsed OK:", len(df) - len(bad_rows), "Parse errors:", len(bad_rows))
if len(bad_rows) > 0:
    print("\nExamples of malformed scp_codes (first 10):")
    for i, example in bad_rows[:10]:
        print(f" row {i} -> {example}")
# Replace original column with parsed
df["scp_codes"] = df["scp_codes_parsed"]
df.drop(columns=["scp_codes_parsed"], inplace=True)


Total rows: 21799
Parsed OK: 21799 Parse errors: 0


In [13]:
# Robust recovery for scp_statements column name mismatch + rebuild labels
import pandas as pd, os, re
from sklearn.preprocessing import LabelEncoder

print("scp_df columns:", scp_df.columns.tolist())
display(scp_df.head(6))

# Heuristics to find code column and diagnostic/class column
cols = [c.lower() for c in scp_df.columns]

# Candidate code columns (common variants)
code_candidates = [c for c in scp_df.columns if re.search(r"scp[_\s]*code|code$|statement|scp_code", c, flags=re.I)]
# Candidate diagnostic class columns
class_candidates = [c for c in scp_df.columns if re.search(r"diagnostic[_\s]*class|diag_class|diagnostic_class|class$", c, flags=re.I)]
# Candidate diagnostic flag column (to filter rows where diagnostic==1)
diagnostic_flag_candidates = [c for c in scp_df.columns if re.search(r"^diagnostic$|^is_diagnostic$|diagnostic_flag", c, flags=re.I)]

print("code_candidates:", code_candidates)
print("class_candidates:", class_candidates)
print("diagnostic_flag_candidates:", diagnostic_flag_candidates)

# Choose columns (best-effort)
code_col = code_candidates[0] if code_candidates else None
class_col = class_candidates[0] if class_candidates else None
diag_flag_col = diagnostic_flag_candidates[0] if diagnostic_flag_candidates else None

# If still None, try some fallbacks by inspecting columns
if code_col is None:
    # try common names
    for alt in ["scp_statement", "scp", "code", "statement"]:
        if alt in scp_df.columns:
            code_col = alt; break

if class_col is None:
    for alt in ["diagnostic_class", "diagnosticclass", "diagnostic-class", "class"]:
        if alt in scp_df.columns:
            class_col = alt; break

print(f"Using code_col = {code_col!r}, class_col = {class_col!r}, diag_flag_col = {diag_flag_col!r}")

# If code_col is still missing, we cannot map; raise informative error
if code_col is None:
    # Show columns again and stop
    raise KeyError("Could not find any SCP code column in scp_statements.csv. Columns are: {}".format(scp_df.columns.tolist()))

# If class_col missing, we'll fallback to identity mapping (scp_code -> scp_code)
if class_col is None:
    print("Warning: no diagnostic-class column found in scp_statements.csv. Falling back to identity mapping (each scp code maps to itself).")
    scp_df["__diag_class_fallback__"] = scp_df[code_col].astype(str)
    class_col = "__diag_class_fallback__"

# If diagnostic flag column exists, filter to rows where diagnostic==1
if diag_flag_col is not None:
    try:
        scp_df_filtered = scp_df[scp_df[diag_flag_col] == 1].copy()
        print(f"Filtered scp_statements using {diag_flag_col}==1 -> {len(scp_df_filtered)} rows")
    except Exception:
        scp_df_filtered = scp_df.copy()
else:
    scp_df_filtered = scp_df.copy()

# Build mapping: scp_code -> diagnostic_class (best-effort)
# Convert both sides to str to avoid dtype mismatches
mapping = {}
for _, row in scp_df_filtered.iterrows():
    key = str(row[code_col])
    val = str(row[class_col])
    mapping[key] = val

print(f"Built mapping for {len(mapping)} scp codes (examples):")
for i, (k,v) in enumerate(mapping.items()):
    if i >= 8: break
    print(" ", k, "->", v)

# Now use mapping to assign diagnostic_class for each record in df
def get_diag_from_scp_dict(scp_dict):
    # scp_dict is a dict of codes->weights (we parsed them)
    if not isinstance(scp_dict, dict) or len(scp_dict) == 0:
        return None
    # find keys that exist in mapping
    candidates = {k: w for k,w in scp_dict.items() if str(k) in mapping}
    if candidates:
        main_code = max(candidates, key=candidates.get)
        return mapping[str(main_code)]
    # no code matches mapping: try match any code as-is (string)
    for k in scp_dict:
        if str(k) in mapping:
            return mapping[str(k)]
    # fallback: return the single highest-weight code name itself (string)
    try:
        return str(max(scp_dict, key=scp_dict.get))
    except Exception:
        return None

df["diagnostic_class"] = df["scp_codes"].apply(get_diag_from_scp_dict)

before = len(df)
df = df.dropna(subset=["diagnostic_class"]).reset_index(drop=True)
after = len(df)
print(f"Dropped {before-after} records with no mapped diagnostic_class; remaining {after}")

# Label encode
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["diagnostic_class"])
class_names = list(le.classes_)
print("Label encoding done. #classes =", len(class_names))
print("Classes (examples):", class_names[:10])

# Done — now you can select folds and continue


scp_df columns: ['Unnamed: 0', 'description', 'diagnostic', 'form', 'rhythm', 'diagnostic_class', 'diagnostic_subclass', 'Statement Category', 'SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code', 'DICOM Code']


,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
2,DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
3,LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
4,NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7
5,IMI,inferior myocardial infarction,1.0,NaN,NaN,MI,IMI,Myocardial Infarction,inferior myocardial infarction,161.0,NaN,NaN,NaN


code_candidates: ['Statement Category', 'SCP-ECG Statement Description', 'AHA code', 'CDISC Code', 'DICOM Code']
class_candidates: ['diagnostic_class', 'diagnostic_subclass']
diagnostic_flag_candidates: ['diagnostic']
Using code_col = 'Statement Category', class_col = 'diagnostic_class', diag_flag_col = 'diagnostic'
Filtered scp_statements using diagnostic==1 -> 44 rows
Built mapping for 11 scp codes (examples):
  other ST-T descriptive statements -> STTC
  Basic roots for coding ST-T changes and abnormalities -> STTC
  Normal/abnormal -> NORM
  Myocardial Infarction -> MI
  Ventricular Hypertrophy -> HYP
  Intraventricular and intra-atrial Conduction disturbances -> CD
  Sinus node dysfunction, atrial and AV-conduction defects -> CD
  ischemic ST-T changes -> STTC
Dropped 0 records with no mapped diagnostic_class; remaining 21799
Label encoding done. #classes = 51
Classes (examples): ['1AVB', '2AVB', '3AVB', 'AFIB', 'AFLT', 'ALMI', 'AMI', 'ASMI', 'CLBBB', 'CRBBB']


In [14]:
df = df[df["diagnostic_class"].notna()].copy()
df.shape


(21799, 30)

In [15]:
df = df.drop_duplicates(subset=["filename_lr"])
df.shape


(21799, 30)

In [16]:
df.isna().sum()


,0
ecg_id,0
patient_id,0
age,0
sex,0
height,14825
weight,12378
nurse,1473
site,17
device,0
recording_date,0


In [17]:
# assume PTBXL_ROOT and df already defined from previous steps
import os
print("Total records:", len(df))
print(df.isna().sum())
print("Example columns present:", list(df.columns)[:40])


Total records: 21799
ecg_id                              0
patient_id                          0
age                                 0
sex                                 0
height                          14825
weight                          12378
nurse                            1473
site                               17
device                              0
recording_date                      0
report                              0
scp_codes                           0
heart_axis                       8468
infarction_stadium1             16187
infarction_stadium2             21696
validated_by                     9378
second_opinion                      0
initial_autogenerated_report        0
validated_by_human                  0
baseline_drift                  20201
static_noise                    18539
burst_noise                     21186
electrodes_problems             21769
extra_beats                     19850
pacemaker                       21508
strat_fold                   

In [18]:
# Drop exact duplicate metadata rows (shouldn't be many)
before = len(df)
df = df.drop_duplicates(subset=["filename_lr"]).reset_index(drop=True)
print("Dropped duplicates:", before - len(df), "Remaining:", len(df))

# Verify each filename_lr corresponds to both .dat and .hea files on disk
missing_files = []
for rel in df["filename_lr"].tolist():
    base = os.path.splitext(os.path.join(PTBXL_ROOT, rel))[0]  # no ext
    if not (os.path.exists(base + ".dat") and os.path.exists(base + ".hea")):
        missing_files.append(base)
print("Records with missing files:", len(missing_files))
# Optionally show examples
missing_files[:10]
# Drop rows that point to missing waveform files (we can't process them)
if missing_files:
    df = df[~df["filename_lr"].apply(lambda fn: os.path.splitext(os.path.join(PTBXL_ROOT, fn))[0] in set(missing_files))].reset_index(drop=True)
    print("After removing missing-file rows:", len(df))


Dropped duplicates: 0 Remaining: 21799
Records with missing files: 301
After removing missing-file rows: 21498


In [19]:
# Columns you probably want to keep for modelling/reporting:
keep_cols = [
    "ecg_id","patient_id","filename_lr","age","sex","report","scp_codes",
    "diagnostic_class","label_id","strat_fold","recording_date","site","device"
]

# Columns to drop because >50% missing or not useful right now (tweak as you like)
drop_candidates = ["height","weight","infarction_stadium1","infarction_stadium2","validated_by","heart_axis"]
# If you want to keep height/weight, skip dropping them — code below will impute instead.
for c in drop_candidates:
    if c in df.columns and df[c].isna().mean() > 0.5:
        if c in df.columns:
            df.drop(columns=[c], inplace=True, errors="ignore")
            print("Dropped column (very sparse):", c)

# But keep height/weight if you prefer: comment the above drop and use imputation instead.
print("Columns remaining:", df.columns.tolist()[:40])


Dropped column (very sparse): height
Dropped column (very sparse): weight
Dropped column (very sparse): infarction_stadium1
Dropped column (very sparse): infarction_stadium2
Columns remaining: ['ecg_id', 'patient_id', 'age', 'sex', 'nurse', 'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis', 'validated_by', 'second_opinion', 'initial_autogenerated_report', 'validated_by_human', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr', 'diagnostic_class', 'label_id']


In [20]:
# Corrected imputation for numeric vs categorical columns (run this cell)
import pandas as pd, numpy as np, os

# Columns we considered before (modify if you want to keep/drop others)
cols_to_check = ["height", "weight", "heart_axis", "infarction_stadium1", "infarction_stadium2", "validated_by", "nurse", "site", "device"]

numeric_cols = []
cat_cols = []
for c in cols_to_check:
    if c not in df.columns:
        continue
    # treat as numeric if dtype is numeric OR all non-missing entries look numeric
    try:
        # attempt to coerce non-null values to numeric
        sample = df[c].dropna().head(200).astype(str)
        # If most of sample elements can be converted to float, treat as numeric
        convertible = sample.apply(lambda s: pd.to_numeric(s, errors='coerce')).notna().sum()
        if convertible >= max(1, int(0.8 * len(sample))):
            numeric_cols.append(c)
        else:
            cat_cols.append(c)
    except Exception:
        cat_cols.append(c)

print("Numeric columns detected:", numeric_cols)
print("Categorical columns detected:", cat_cols)

# Impute numeric columns with median + create missing flags
for col in numeric_cols:
    try:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        med = float(df[col].median(skipna=True))
        df[f"{col}_missing"] = df[col].isna().astype(int)
        df[col] = df[col].fillna(med)
        print(f"Imputed numeric {col} with median={med:.3f}")
    except Exception as e:
        print("Could not impute numeric col", col, e)

# Fill categorical columns with 'unknown' and create missing flags
for col in cat_cols:
    df[f"{col}_missing"] = df[col].isna().astype(int)
    # convert entire column to string and fill
    df[col] = df[col].astype(str).fillna("unknown").replace("nan", "unknown")
    df[col] = df[col].replace({"None":"unknown","NoneType":"unknown"})
    print(f"Filled categorical {col} missing values with 'unknown'; unique values (first 20):")
    try:
        print(pd.Series(df[col].unique()).astype(str)[:20].tolist())
    except Exception:
        print(" (unable to list unique values)")

# Special note for heart_axis — show summary counts to help decide mapping
if "heart_axis" in df.columns:
    print("\nheart_axis value counts (top 30):")
    print(df["heart_axis"].value_counts(dropna=False).head(30))

# Save cleaned metadata preview (optional)
preview_path = os.path.join("/content/drive/MyDrive/PTBXL", "ptbxl_metadata_after_impute_preview.csv")
df.head(200).to_csv(preview_path, index=False)
print("\nSaved preview of first 200 rows to:", preview_path)


Numeric columns detected: ['validated_by', 'nurse', 'site']
Categorical columns detected: ['heart_axis', 'device']
Imputed numeric validated_by with median=1.000
Imputed numeric nurse with median=1.000
Imputed numeric site with median=1.000
Filled categorical heart_axis missing values with 'unknown'; unique values (first 20):
['unknown', 'LAD', 'ALAD', 'RAD', 'AXR', 'MID', 'ARAD', 'AXL', 'SAG']
Filled categorical device missing values with 'unknown'; unique values (first 20):
['CS-12   E', 'AT-6 C 5.0', 'AT-6 C', 'CS-12', 'AT-6 C 5.5', 'AT-6 C 5.8', 'AT-6 C 5.6', 'AT-6     6', 'AT-6 C 5.3', 'AT-60    3', 'CS100    3']

heart_axis value counts (top 30):
heart_axis
unknown    8321
MID        7606
LAD        3712
ALAD       1365
RAD         221
ARAD        118
AXL         101
AXR          51
SAG           3
Name: count, dtype: int64

Saved preview of first 200 rows to: /content/drive/MyDrive/PTBXL/ptbxl_metadata_after_impute_preview.csv


In [21]:
# Normalize sex column
if "sex" in df.columns:
    df["sex"] = df["sex"].astype(str).str.upper().str.strip().replace({"M":"M","F":"F","MALE":"M","FEMALE":"F"})
    df["sex"] = df["sex"].where(df["sex"].isin(["M","F"]), other="U")  # U = unknown/other

# Age: convert to numeric & clip unrealistic values
if "age" in df.columns:
    df["age"] = pd.to_numeric(df["age"], errors="coerce")
    df["age_missing"] = df["age"].isna().astype(int)
    df["age"] = df["age"].fillna(df["age"].median())
    df["age"] = df["age"].clip(lower=0, upper=120)


In [22]:
# Example: convert recording_date to datetime and extract year/month
if "recording_date" in df.columns:
    df["recording_date"] = pd.to_datetime(df["recording_date"], errors="coerce")
    df["recording_year"] = df["recording_date"].dt.year.fillna(0).astype(int)
    df["recording_month"] = df["recording_date"].dt.month.fillna(0).astype(int)


In [23]:
CLEAN_META_PATH = "/content/drive/MyDrive/PTBXL/ptbxl_database_cleaned.csv"
df.to_csv(CLEAN_META_PATH, index=False)
print("Saved cleaned metadata to:", CLEAN_META_PATH)


Saved cleaned metadata to: /content/drive/MyDrive/PTBXL/ptbxl_database_cleaned.csv


In [24]:
CLEAN_META_PATH = "/content/drive/MyDrive/PTBXL/ptbxl_database_cleaned.csv"
df.to_csv(CLEAN_META_PATH, index=False)
print("Saved cleaned metadata to:", CLEAN_META_PATH)


Saved cleaned metadata to: /content/drive/MyDrive/PTBXL/ptbxl_database_cleaned.csv


In [25]:
import wfdb, numpy as np
from scipy.signal import butter, filtfilt

TARGET_FS = 100  # records100
TARGET_SECONDS = 10  # full length
TARGET_LEN = TARGET_FS * TARGET_SECONDS  # 1000 for 10s? wait -> corrected: 100Hz * 10s = 1000
# Note: 100Hz * 10s = 1000 samples. (If you opted 5s, set TARGET_SECONDS=5 -> 500)

def bandpass(sig, low=0.5, high=40.0, fs=TARGET_FS, order=4):
    b,a = butter(order, [low/(fs/2), high/(fs/2)], btype="band")
    return filtfilt(b,a,sig,axis=0)

def load_and_preprocess(rec_rel_path, crop_samples=None, do_filter=True):
    recpath = os.path.splitext(os.path.join(PTBXL_ROOT, rec_rel_path))[0]
    sig, _ = wfdb.rdsamp(recpath, sampfrom=0, sampto=crop_samples)
    sig = sig.astype(np.float32)
    # keep first 12 channels if extra (PTB-XL has 12)
    if sig.ndim==2 and sig.shape[1] >= 12:
        sig = sig[:, :12]
    else:
        raise ValueError("Bad signal shape for "+rec_rel_path)
    # pad/trim to TARGET_LEN
    L = sig.shape[0]
    if L < TARGET_LEN:
        pad = TARGET_LEN - L
        sig = np.vstack([sig, np.zeros((pad, sig.shape[1]), dtype=np.float32)])
    elif L > TARGET_LEN:
        sig = sig[:TARGET_LEN, :]
    # optional filter
    if do_filter:
        try:
            sig = bandpass(sig, low=0.5, high=40, fs=TARGET_FS)
        except Exception as e:
            # if filter fails, keep original signal (but flag)
            print("Filter failed for", rec_rel_path, e)
    # per-record per-lead standardize
    mean = sig.mean(axis=0, keepdims=True)
    std = sig.std(axis=0, keepdims=True) + 1e-8
    sig = (sig - mean) / std
    return sig  # shape (TARGET_LEN, 12)


In [27]:
# Robust Step 9: Process Training Data
import os, numpy as np, traceback
from tqdm import tqdm

# ---- CONFIGURE ----
# CHANGE: We set this to "train" to generate the training dataset
SPLIT_TO_PROCESS = "train"
DO_BANDPASS = True
TARGET_FS = 100
TARGET_SECONDS = 10
TARGET_LEN = TARGET_FS * TARGET_SECONDS
OUT_DIR = "/content/drive/MyDrive/PTBXL"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- Ensure splits exist ----
if 'df' not in globals():
    raise RuntimeError("df is not defined. Run earlier cells that load metadata and mapping first.")

if not ("strat_fold" in df.columns):
    # If folds are missing, we assign them now (Standard PTB-XL split)
    print("Assigning folds...")
    df['strat_fold'] = df['strat_fold'].fillna(10).astype(int)

# Build train/val/test if not present
if 'train_df' not in globals():
    train_df = df[df["strat_fold"].between(1,8)].reset_index(drop=True)
    val_df   = df[df["strat_fold"] == 9].reset_index(drop=True)
    test_df  = df[df["strat_fold"] == 10].reset_index(drop=True)
    print("Built splits: Train/Val/Test sizes =", len(train_df), len(val_df), len(test_df))

# Pick dataframe
if SPLIT_TO_PROCESS == "test":
    process_df = test_df.copy()
elif SPLIT_TO_PROCESS == "train":
    process_df = train_df.copy()
elif SPLIT_TO_PROCESS == "val":
    process_df = val_df.copy()
else:
    process_df = df.copy()

print(f"Processing split: {SPLIT_TO_PROCESS} (Records: {len(process_df)})")

# ---- Helper functions ----
import wfdb
from scipy.signal import butter, filtfilt

def bandpass(sig, low=0.5, high=40.0, fs=TARGET_FS, order=4):
    b,a = butter(order, [low/(fs/2), high/(fs/2)], btype="band")
    return filtfilt(b,a,sig,axis=0)

def rel_to_recpath(root, rel):
    return os.path.splitext(os.path.join(root, rel))[0]

def load_and_preprocess(rel_path, root, crop_samples=None, do_filter=DO_BANDPASS):
    recpath_no_ext = rel_to_recpath(root, rel_path)
    sig, _ = wfdb.rdsamp(recpath_no_ext, sampfrom=0, sampto=crop_samples)
    sig = sig.astype(np.float32)
    if sig.ndim != 2 or sig.shape[1] < 12:
        raise ValueError(f"Bad shape: {sig.shape}")
    sig = sig[:, :12]  # keep 12 leads
    L = sig.shape[0]
    if L < TARGET_LEN:
        pad = TARGET_LEN - L
        sig = np.vstack([sig, np.zeros((pad, sig.shape[1]), dtype=np.float32)])
    elif L > TARGET_LEN:
        sig = sig[:TARGET_LEN, :]
    if do_filter:
        try:
            sig = bandpass(sig, low=0.5, high=40, fs=TARGET_FS)
        except:
            pass
    mean = sig.mean(axis=0, keepdims=True)
    std = sig.std(axis=0, keepdims=True) + 1e-8
    sig = (sig - mean) / std
    return sig, recpath_no_ext

# ---- Loop ----
X_list = []
y_list = []
rec_paths = []
failed = []

for idx, row in tqdm(process_df.iterrows(), total=len(process_df), desc=f"Processing {SPLIT_TO_PROCESS}"):
    try:
        rel = row["filename_lr"]
        sig, rec_no_ext = load_and_preprocess(rel, PTBXL_ROOT, crop_samples=None, do_filter=DO_BANDPASS)
        X_list.append(sig)
        y_list.append(int(row["label_id"]))
        rec_paths.append(rec_no_ext)
    except Exception as e:
        failed.append(str(e))
        continue

if len(X_list) == 0:
    raise RuntimeError("No records processed. Check paths.")

X = np.stack(X_list, axis=0).astype(np.float32)
y = np.array(y_list, dtype=np.int32)

# ---- Save ----
out_file = os.path.join(OUT_DIR, f"ptbxl_{SPLIT_TO_PROCESS}_preprocessed_100hz_{TARGET_SECONDS}s.npz")

# Use class_names if defined, otherwise generic
if 'class_names' not in globals():
    class_names = [str(i) for i in np.unique(y)]

np.savez_compressed(out_file, X=X, y=y, class_names=np.array(class_names, dtype=object))
print(f"SUCCESS! Saved {SPLIT_TO_PROCESS} data to: {out_file}")

Processing split: train (Records: 17166)


Processing train: 100%|██████████| 17166/17166 [2:53:57<00:00,  1.64it/s]


SUCCESS! Saved train data to: /content/drive/MyDrive/PTBXL/ptbxl_train_preprocessed_100hz_10s.npz
